In [1]:
!pip install tabgan

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.9/81.9 kB 6.8 MB/s eta 0:00:00


In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

import torch.optim as optim

In [3]:
from tabgan.sampler import OriginalGenerator, GANGenerator

# import libraries
import pandas as pd
import numpy as np
from scipy import stats

import matplotlib.pyplot as plt
import sklearn as sklearn
import seaborn as sns

# for models
import xgboost as xgb
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.metrics import  f1_score, classification_report
from sklearn.feature_selection import SelectFromModel

from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.inspection import permutation_importance
from xgboost import plot_importance

from sklearn.linear_model import LogisticRegression

# display all the columns and rows
pd.set_option('display.max_rows', 300)
pd.set_option('display.max_colwidth', None)
# set figure size for single graphs
plt.rcParams['figure.figsize'] = [15, 6]

/usr/local/lib/python3.10/dist-packages/_ctgan/synthesizer.py:7: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [4]:
df = pd.read_csv('/content/drive/MyDrive/Course Work 1/final_df.csv')
df

,loan_amnt,term,int_rate,sub_grade,emp_length,home_ownership,annual_inc,verification_status,loan_status,purpose,...,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,num_accts_ever_120_pd,num_actv_rev_tl,num_bc_tl,num_il_tl,pct_tl_nvr_dlq,total_bc_limit,fico_diff
0,12000.0,36 months,7.97,4.0,10+ years,OWN,42000.0,Source Verified,Fully Paid,debt_consolidation,...,1.0,0.0,14.0,0.0,6.0,2.0,7.0,100.0,15500.0,4.0
1,10000.0,36 months,9.44,5.0,3 years,MORTGAGE,55000.0,Not Verified,Fully Paid,debt_consolidation,...,10.0,2.0,49.0,0.0,2.0,1.0,5.0,100.0,10500.0,4.0
2,12800.0,36 months,13.59,11.0,5 years,RENT,90000.0,Not Verified,Fully Paid,debt_consolidation,...,5.0,0.0,5.0,0.0,6.0,6.0,15.0,83.0,14750.0,4.0
3,15000.0,36 months,13.59,11.0,4 years,MORTGAGE,180000.0,Source Verified,Fully Paid,medical,...,13.0,4.0,16.0,0.0,14.0,16.0,17.0,100.0,116700.0,4.0
4,40000.0,60 months,7.97,4.0,10+ years,OWN,200000.0,Verified,Charged Off,home_improvement,...,0.0,6.0,28.0,0.0,6.0,10.0,35.0,100.0,48400.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
903811,24000.0,60 months,23.99,21.0,< 1 year,RENT,107000.0,Source Verified,Charged Off,other,...,4.0,0.0,18.0,1.0,7.0,11.0,5.0,87.5,28200.0,4.0
903812,10000.0,36 months,7.99,4.0,10+ years,MORTGAGE,65000.0,Source Verified,Fully Paid,debt_consolidation,...,1.0,0.0,1.0,0.0,8.0,12.0,4.0,92.6,34800.0,4.0
903813,10050.0,36 months,16.99,15.0,8 years,RENT,37000.0,Not Verified,Charged Off,debt_consolidation,...,2.0,0.0,6.0,0.0,13.0,4.0,3.0,95.2,9000.0,4.0
903814,6000.0,36 months,11.44,8.0,5 years,RENT,41000.0,Source Verified,Fully Paid,credit_card,...,12.0,0.0,32.0,0.0,3.0,4.0,9.0,94.4,8100.0,4.0


In [5]:
# One-hot encoding with Pandas: setting the dummies
df_xgb = pd.get_dummies(df, columns=['application_type',
                                    'addr_state',
                                    'purpose',
                                    'verification_status',
                                    'home_ownership',
                                    'emp_length',
                                    'term'])
# reset index
df_xgb= df_xgb.reset_index(drop=True)
df_xgb = df_xgb.replace({"loan_status":{ "Fully Paid": 0, "Charged Off": 1}})

In [6]:
df_nn = df_xgb.sample(100000, random_state=2023)
df_nn_sample = df_nn.sample(10000, random_state=2023)

In [22]:
x_train, x_test, y_train, y_test = train_test_split(df_nn_sample.drop('loan_status', axis=1),
                                                                            df_nn_sample['loan_status'], test_size=0.3,
                                                        random_state=2023, stratify=df_nn_sample['loan_status'])

In [17]:
# generate data
#new_train1, new_target1 = OriginalGenerator().generate_data_pipe(x_train, pd.DataFrame(y_train), x_test, )
new_train2, new_target2 = GANGenerator(gen_x_times=1.1, cat_cols=None,
                                      bot_filter_quantile=0.9, top_filter_quantile=0.999,
                                       is_post_process=True).generate_data_pipe(x_train.iloc[:100],
                                                                                pd.DataFrame(y_train.iloc[:100]), x_test.iloc[:100],
                                                                                only_adversarial=False, use_adversarial=True)

Fitting CTGAN transformers for each column:   0%|          | 0/125 [00:00<?, ?it/s]

Training CTGAN, epochs::   0%|          | 0/500 [00:00<?, ?it/s]

ValueError: ignored

In [29]:
from ctgan import CTGAN

In [36]:
discrete_columns = list(df_nn_sample.select_dtypes(include=['int', 'uint8']).columns)

In [40]:
ctgan = CTGAN(epochs=20)
ctgan.fit(df_nn, discrete_columns)

# Create synthetic data
synthetic_data = ctgan.sample(len(df_nn))

In [42]:
synthetic_data.to_csv('/content/drive/MyDrive/Course Work 1/CTGAN_df.csv')

In [43]:
synthetic_data.loan_status.value_counts()

0    92514
1     7486
Name: loan_status, dtype: int64